In [ ]:
import tensorflow as tf
tf.config.list_physical_devices('GPU')

from google.colab import drive
drive.mount('/content/drive')

!ls /content/drive/MyDrive

!ls "/content/drive/MyDrive/waste and wildlife images"


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
'Colab Notebooks'
'Copy of [EXT] GDG on Campus - TechSprint Slides Template.gslides'
'DocScanner 13-Aug-2024 8-11 pm (1).jpg'
'DocScanner 13-Aug-2024 8-11 pm.jpg'
 is.pdf
'waste and wildlife images'
garbage  test1.jpg  test.jpg  wildlife


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

DATASET_PATH = "/content/drive/MyDrive/waste and wildlife images"

datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='training'
)

val_data = datagen.flow_from_directory(
    DATASET_PATH,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)


Found 4983 images belonging to 2 classes.
Found 1245 images belonging to 2 classes.


In [ ]:
import tensorflow as tf

base_model = tf.keras.applications.MobileNetV2(
    input_shape=(224, 224, 3),
    include_top=False,
    weights='imagenet'
)

base_model.trainable = False  # freeze pretrained layers

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(
    optimizer='adam',
    loss='binary_crossentropy',
    metrics=['accuracy']
)

model.summary()


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ mobilenetv2_1.00_224            │ (None, 7, 7, 1280)     │     2,257,984 │
│ (Functional)                    │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 1280)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 1)              │         1,281 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,259,265 (8.62 MB)

 Trainable params: 1,281 (5.00 KB)

 Non-trainable params: 2,257,984 (8.61 MB)

In [ ]:
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=5
)


Epoch 1/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 85s 238ms/step - accuracy: 0.9509 - loss: 0.1224 - val_accuracy: 0.9984 - val_loss: 0.0084
Epoch 2/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 33s 107ms/step - accuracy: 0.9991 - loss: 0.0062 - val_accuracy: 1.0000 - val_loss: 0.0050
Epoch 3/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 34s 108ms/step - accuracy: 1.0000 - loss: 0.0031 - val_accuracy: 1.0000 - val_loss: 0.0022
Epoch 4/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 34s 108ms/step - accuracy: 1.0000 - loss: 0.0014 - val_accuracy: 1.0000 - val_loss: 0.0017
Epoch 5/5
312/312 ━━━━━━━━━━━━━━━━━━━━ 34s 108ms/step - accuracy: 1.0000 - loss: 0.0012 - val_accuracy: 1.0000 - val_loss: 0.0015


In [ ]:
import numpy as np
from tensorflow.keras.preprocessing import image

def aquatic_decision_system(img_path):
    img = image.load_img(img_path, target_size=(224,224))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)[0][0]

    if prediction < 0.5:
        return "Waste Detected → COLLECT"
    else:
        return "Natural Object → RELEASE"


In [ ]:
!ls /content/drive/MyDrive


'Colab Notebooks'
'Copy of [EXT] GDG on Campus - TechSprint Slides Template.gslides'
'DocScanner 13-Aug-2024 8-11 pm (1).jpg'
'DocScanner 13-Aug-2024 8-11 pm.jpg'
 is.pdf
'waste and wildlife images'


In [ ]:
print(
    aquatic_decision_system(
        "/content/drive/MyDrive/waste and wildlife images/test1.jpg"
    )
)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 35ms/step
Natural Object → RELEASE


In [ ]:
!pip install gradio


In [ ]:
import gradio as gr
import tempfile
from PIL import Image

def gradio_predict(img):
    # Save uploaded image temporarily
    temp = tempfile.NamedTemporaryFile(suffix=".jpg", delete=False)
    img.save(temp.name)

    return aquatic_decision_system(temp.name)


In [ ]:
interface = gr.Interface(
    fn=gradio_predict,
    inputs=gr.Image(type="pil", label="Upload Aquatic Image"),
    outputs=gr.Textbox(label="Decision"),
    title="AI-Based Aquatic Waste Classification",
    description="Upload an image to classify it as Waste or Natural/Wildlife"
)

interface.launch()


It looks like you are running Gradio on a hosted Jupyter notebook, which requires `share=True`. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://da6d1ef2ce26456085.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
